# Hacking the Kidney: Inference of Unet model

References:

- [256x256 images](https://www.kaggle.com/iafoss/256x256-images)

- [Making a successful submission](https://www.kaggle.com/igor14497/making-a-successful-submission])

- [HuBMAP: TF with TPU EfficientUNet 512x512[subm]](https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-subm)

- [Global Mask Shift](https://www.kaggle.com/tivfrvqhs5/global-mask-shift)

Many thanks to that notebooks' authors.

This notebook makes an inference for Unet model with EfficientNet backbone based on [this library](https://github.com/qubvel/segmentation_models). The code for training models is [here](https://github.com/vgarshin/kaggle_kidney/blob/master/kidney_train.ipynb).

In [ ]:
!pip install ../input/keras-applications/Keras_Applications-1.0.8/ -f ./ --no-index
!pip install ../input/image-classifiers/image_classifiers-1.0.0/ -f ./ --no-index
!pip install ../input/efficientnet-1-0-0/efficientnet-1.0.0/ -f ./ --no-index
!pip install ../input/segmentation-models/segmentation_models-1.0.1/ -f ./ --no-index

In [ ]:
%env SM_FRAMEWORK=tf.keras

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import gc
import cv2
import sys
import json
import time
import pickle
import shutil
import numba
import numpy as np
import pandas as pd 
import tifffile as tiff
import rasterio
from rasterio.windows import Window
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import *
import segmentation_models as sm
from segmentation_models import Unet, FPN, Linknet
from segmentation_models.losses import bce_jaccard_loss
from tqdm import tqdm
print('tensorflow version:', tf.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for gpu_device in gpu_devices:
        print('device available:', gpu_device)
pd.set_option('display.max_columns', None)

In [ ]:
TEST = True
KAGGLE = True
VER = 'v46'
if KAGGLE:
    DATA_PATH = '../input/hubmap-kidney-segmentation'
    MDLS_PATH = f'../input/kidney-models-{VER}'
else:
    DATA_PATH = './data'
    MDLS_PATH = f'./models_{VER}'
THRESHOLD = .3
VOTERS = 1
TTAS = [0, 1, 2, 3]
FOLDS = [1, 2]
EXPAND = 4
MIN_OVERLAP = 512
IDNT = rasterio.Affine(1, 0, 0, 0, 1, 0)
STRATEGY = tf.distribute.get_strategy() 
SUB_PATH = f'{DATA_PATH}/test' if TEST else f'{DATA_PATH}/train'

start_time = time.time()

In [ ]:
with open(f'{MDLS_PATH}/params.json') as file:
    params = json.load(file)
if 'umodel' not in params.keys(): params['umodel'] = 'unet'
print('loaded params:', params)

In [ ]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s) // 2):
            start = int(s[2 * i]) - 1
            length = int(s[2 * i + 1])
            img[start : start + length] = 1 + m
    return img.reshape(shape).T

def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth=1):
    return (1 - dice_coef(y_true, y_pred, smooth))

def bce_dice_loss(y_true, y_pred):
    return params['bce_weight'] * binary_crossentropy(y_true, y_pred) + \
        (1 - params['bce_weight']) * dice_loss(y_true, y_pred)

def get_model(backbone, input_shape, loss_type='bce_dice', 
              umodel='unet', classes=1, lr=.001):
    if backbone == 'efficientnetb0':
        weights = f'{MDLS_PATH}/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
    elif backbone == 'efficientnetb1':
        weights = f'{MDLS_PATH}/efficientnet-b1_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
    elif backbone == 'efficientnetb2':
        weights = f'{MDLS_PATH}/efficientnet-b2_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
    elif backbone == 'efficientnetb3':
        weights = f'{MDLS_PATH}/efficientnet-b3_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
    elif backbone == 'resnet34':
        weights = f'{MDLS_PATH}/resnet34_imagenet_1000_no_top.h5'
    else:
        raise AttributeError('mode parameter error')
    with STRATEGY.scope():
        if loss_type == 'bce_dice': 
            loss = bce_dice_loss
        elif loss_type == 'bce_jaccard_loss':
            loss = bce_jaccard_loss
        else:
            raise AttributeError('loss mode parameter error')
        if umodel == 'unet':
            model = Unet(backbone_name=backbone, encoder_weights=weights,
                         input_shape=input_shape,
                         classes=classes, activation='sigmoid')
        elif umodel == 'fpn':
            model = FPN(backbone_name=backbone, encoder_weights=weights,
                        input_shape=input_shape,
                        classes=classes, activation='sigmoid')
        elif umodel == 'link':
            model = Linknet(backbone_name=backbone, encoder_weights=weights,
                            input_shape=input_shape,
                            classes=classes, activation='sigmoid')
        else:
            raise AttributeError('umodel mode parameter error')
        model.compile(
            optimizer=tfa.optimizers.Lookahead(
                tf.keras.optimizers.Adam(learning_rate=lr)
            ),
            loss=loss, 
            metrics=[dice_coef]
        )
    return model

In [ ]:
def make_grid(shape, window=256, min_overlap=32):
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx, ny, 4), dtype=np.int64) 
    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx * ny, 4)

def flip(img, axis=0):
    if axis == 1:
        return img[::-1, :, ]
    elif axis == 2:
        return img[:, ::-1, ]
    elif axis == 3:
        return img[::-1, ::-1, ]
    else:
        return img

In [ ]:
img_files = [x for x in os.listdir(SUB_PATH) if '.tiff' in x]
print('images idxs:', img_files)

In [ ]:
subm = {}
for i_img, img_file in enumerate(img_files):
    print('-' * 20, img_file, '-' * 20)
    img_data = rasterio.open(os.path.join(SUB_PATH, img_file), transform=IDNT)
    print('img shape:', img_data.shape)
    if img_data.count != 3:
        print('img file with subdatasets as channels')
        layers = [rasterio.open(subd) for subd in img_data.subdatasets]
    img_preds = np.zeros(img_data.shape, dtype=np.uint8)
    tile_size = int(params['img_size'] * EXPAND)
    tile_resized = int(tile_size * params['resize'])
    slices = make_grid(
        img_data.shape, 
        window=tile_resized, 
        min_overlap=MIN_OVERLAP
    )
    models = []
    folds = FOLDS
    for n_fold in folds:
        checkpoint_path = f'{MDLS_PATH}/model_{n_fold}.hdf5'
        model = get_model(
            params['backbone'], 
            input_shape=(tile_size, tile_size, 3),
            loss_type=params['loss'],
            umodel=params['umodel']
        )
        model.load_weights(checkpoint_path)
        models.append(model)
    for (x1, x2, y1, y2) in tqdm(slices, desc=f'{img_file}'):
        if img_data.count == 3: # normal
            img = img_data.read(
                [1, 2, 3], 
                window=Window.from_slices((x1, x2), (y1, y2))
            )
            img = np.moveaxis(img, 0, -1)
        else: # with subdatasets/layers
            img = np.zeros((tile_resized, tile_resized, 3), dtype=np.uint8)
            for fl in range(3):
                img[:, :, fl] = layers[fl].read(
                    window=Window.from_slices((x1, x2), (y1, y2))
                )
        img = cv2.resize(img, (tile_size, tile_size))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        pred = np.zeros((tile_size, tile_size), dtype=np.float32)
        for tta_mode in TTAS:
            img_aug = flip(img, axis=tta_mode)
            img_aug = np.expand_dims(img_aug, 0)
            img_aug = img_aug.astype(np.float32) / 255
            pred_aug = np.zeros((tile_size, tile_size), dtype=np.float32)
            for model in models:
                pred_aug += np.squeeze(model.predict(img_aug)) / len(models)
            pred += flip(pred_aug, axis=tta_mode) / len(TTAS)
        pred = cv2.resize(pred, (tile_resized, tile_resized))
        img_preds[x1:x2, y1:y2] = img_preds[x1:x2, y1:y2] + \
            (pred > THRESHOLD).astype(np.uint8)
    del model, models, img, pred, img_aug, pred_aug; gc.collect()
    print('img max:', np.max(img_preds), '| voters:', VOTERS)
    img_preds = (img_preds >= VOTERS).astype(np.uint8)
    rle_pred = rle_encode_less_memory(img_preds)
    subm[i_img] = {'id':img_file.replace('.tiff', ''), 'predicted': rle_pred}
    del img_preds, img_data, rle_pred; gc.collect()

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
df_sub = pd.DataFrame(subm).T
df_sub

In [ ]:
df_sub.to_csv('submission.csv', index=False)